![Title](images/title-page.png)

![Title](images/title-qr.png)

### What is the TPC?
The TPC is a non-profit corporation focused on developing data-centric benchmark standards and disseminating objective, verifiable data to the industry.

### What is TPC-DI?
The TPC-DI benchmark combines and transforms data extracted from an On-Line Transaction Processing (OTLP) system along with other sources of data, and loads it into a data warehouse.

![ETL Diagram](images/tpc-di-etl-diagram.png)

TPC-DI provides `DIGen.jar` to generate the source files.

The JAR is dated and requires a 1.8 JDK.

In [28]:
!jenv local 1.8
!java -jar ~/dev/Tools/DIGen.jar --help

usage: DIGen
 -h                   print this message
 -jvm <JVM options>   JVM options. E.g. -jvm "-Xms1g -Xmx2g"
 -o <directory>       Specify output directory.  Default is output.
 -sf <sf>             Scale factor.  Default value is 5. (range: 3 -
                      2147483647
 -v                   print DIGen version


In [31]:
!rm -rf ~/dev/tpcdi-output
!mkdir -p ~/dev/tpcdi-output
!cd ~/dev/Tools && java -jar ~/dev/Tools/DIGen.jar -o ~/dev/tpcdi-output -sf 5

/Users/stewartbryson/dev/tpcdi-output
########################################################################################################################
                                                  PDGF v2.5_#1343_b4177
                                            Parallel Data Generation Framework
                (c)bankmark UG (haftungsbeschraenkt), Frank M., Danisch M., Rabl T. http://www.bankmark.de
########################################################################################################################
                                                   License information
                            The Software is provided to you as part of the TPC Benchmark DI. 
 When using this software you must agree to the license provided in LICENSE.TXT of this package. Use is restricted to TPC
DI benchmarking purposes as specified in LICENSE.TXT. If you would like to use the software for other purposes, you must
contact bankmark UG (haftungsbeschraenkt) (http://www.

### The GitHub repository has a prebuilt CLI for easily loading the files.
### https://github.com/stewartbryson/dbt-tpcdi

In [33]:
!python tpcdi.py --help

                                                                                
 Usage: tpcdi.py [OPTIONS] COMMAND [ARGS]...                                    
                                                                                
 A utility for loading TPC-DI generated files into Snowflake.                   
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --install-completion        [bash|zsh|fish|powershe  Install completion for  │
│                             ll|pwsh]                 the specified shell.    │
│                                                      [default: None]         │
│ --show-completion           [bash|zsh|fish|powershe  Show completion for the │
│                             ll|pwsh]                 specified shell, to     │
│                                                      copy it or customize    │
│                           

In [40]:
!python tpcdi.py process-files --help

                                                                                
 Usage: tpcdi.py process-files [OPTIONS]                                        
                                                                                
 Upload a file or files into the stage and build the dependent tables.          
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ *  --output-directory                        TEXT     The output directory   │
│                                                       from the TPC-DI        │
│                                                       DIGen.jar execution.   │
│                                                       [default: None]        │
│                                                       [required]             │
│    --file-name                               TEXT     The TPC-DI file name   │
│                           

In [6]:
!python tpcdi.py process-files --output-directory ~/dev/tpcdi-output --file-name DailyMarket.txt --show

File DailyMarket.txt: SKIPPED
----------------------------------------------------------------------------------
|"DM_DATE"   |"DM_S_SYMB"      |"DM_CLOSE"  |"DM_HIGH"  |"DM_LOW"  |"DM_VOL"     |
----------------------------------------------------------------------------------
|2015-07-06  |AAAAAAAAAAAAERN  |242.93      |284.42     |185.08    |111904727.0  |
|2015-07-06  |AAAAAAAAAAAAEYJ  |445.46      |522.3      |386.48    |78849320.0   |
|2015-07-06  |AAAAAAAAAAAAEVC  |910.59      |1148.89    |723.37    |807515829.0  |
|2015-07-06  |AAAAAAAAAAAACEZ  |647.07      |756.68     |473.3     |693226268.0  |
|2015-07-06  |AAAAAAAAAAAADOY  |385.01      |564.67     |295.63    |34628570.0   |
|2015-07-06  |AAAAAAAAAAAADSD  |28.01       |34.59      |23.66     |47032973.0   |
|2015-07-06  |AAAAAAAAAAAAELH  |186.85      |249.13     |170.26    |79305649.0   |
|2015-07-06  |AAAAAAAAAAAAAXX  |880.03      |990.35     |727.51    |353491380.0  |
|2015-07-06  |AAAAAAAAAAAABVO  |911.31      |1143.78    |

If you get nothing else from this video, know that there's an easy way to load this dataset into Snowflake.

But I also wanted to show some interesting approaches using Snowpark.

All of the code samples below are snippets from the CLI with abstractions removed.

We start with a `credentials.json` file to store our Snowflake credentials. Something like this:

```json
{
    "account": "myaccount",
    "user": "myuser",
    "password": "mypassword",
    "role": "myrole",
    "warehouse": "stewart_dev",
    "database": "tpc_di",
    "schema": "digen"
}
```

Then we can make a connection to Snowflake.

In [1]:
import sys, json
from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import *
from pathlib import Path

# Read the credentials.json file
with open("credentials.json") as jsonfile:
    credentials_dict = json.load(jsonfile)

# build the session
session = (
    Session
    .builder
    .configs(credentials_dict)
    .create()
)

Most of the files generated by `DIGen.jar` are pipe-separated files, very similar to CSV files.

These are very simple to handle. First let's upload the file to a stage:

In [15]:
# File paths
source_path = '/Users/stewartbryson/dev/tpcdi-output/Batch1'
stage_path = "@tpcdi/Batch1"

# Put the file
put_result = (
    session
    .file
    .put(
        f"{source_path}/DailyMarket.txt",
        f"{stage_path}/DailyMarket.txt",
        parallel=4,
        auto_compress=True,
    )
)
for result in put_result:
    print(f"File {result.source}: {result.status}")

File DailyMarket.txt: SKIPPED


And now we'll create a table from that file:

In [16]:
# Define the schema
schema = StructType([
                StructField("DM_DATE", DateType(), False),
                StructField("DM_S_SYMB", StringType(), False),
                StructField("DM_CLOSE", FloatType(), False),
                StructField("DM_HIGH", FloatType(), False),
                StructField("DM_LOW", FloatType(), False),
                StructField("DM_VOL", FloatType(), False),
        ])

# create a table from a DataFrame
df = (
    session
    .read
    .schema(schema)
    .option("field_delimiter", '|')
    .csv(f"{stage_path}/DailyMarket.txt")
    .write
    .mode("overwrite")
    .save_as_table('daily_market')
)

# show the table
df = (
    session 
    .table('daily_market') 
    .show()
)


----------------------------------------------------------------------------------
|"DM_DATE"   |"DM_S_SYMB"      |"DM_CLOSE"  |"DM_HIGH"  |"DM_LOW"  |"DM_VOL"     |
----------------------------------------------------------------------------------
|2016-12-18  |AAAAAAAAAAAABZF  |961.07      |1394.14    |847.64    |709969048.0  |
|2016-12-18  |AAAAAAAAAAAABGY  |134.65      |173.1      |106.55    |122085128.0  |
|2016-12-18  |AAAAAAAAAAAACFY  |497.2       |741.06     |427.71    |8263059.0    |
|2016-12-18  |AAAAAAAAAAAACJW  |697.83      |988.37     |630.96    |576146934.0  |
|2016-12-18  |AAAAAAAAAAAAELN  |512.4       |699.93     |508.04    |532344015.0  |
|2016-12-18  |AAAAAAAAAAAADQU  |115.56      |146.54     |71.32     |808265496.0  |
|2016-12-18  |AAAAAAAAAAAABSG  |38.99       |39.65      |29.36     |696226368.0  |
|2016-12-18  |AAAAAAAAAAAAAOO  |640.01      |905.5      |581.14    |828920058.0  |
|2016-12-18  |AAAAAAAAAAAACYZ  |396.78      |536.03     |233.16    |713253731.0  |
|201

The `DIGen.jar` utility generates a series of "finwire" files.

These files represent market history over time.

They are fixed-width, multi-format files.
For instance, the following sample has one of each type of record: `FIN`, `SEC`, and `CMP`:

In [52]:
!cat devrel/multi-record.txt

20151230-152248FIN201542015100120151230    4880880089.63    2473473307.30        4.82        4.42        0.51     200321223.47  139284472514.02    9402305760.19    512872010    5597926720000001595
20151230-152511SECAAAAAAAAAAAAKVDPREF_AACTVDJBJXyQHLBvn EEOGAOvUNgL XwrOxQUBMrgPv                                AMEX  982113436    1903022619730704        1.200000000254
20151230-163207CMPWWfcsOHprIDIUsPfRLrcLPlxaQ                                  0000004432ACTVMCA   1873092521088 Vessey Crescent                                                                                                                                           M5D 1Z1     Winnipeg                 AL                  United States of AmericaMoreno                                        rlRIDCNz dVGrEzomCXIvZVZzFzxCzbGYIEbAXJMJlsYUQEV

We'll start by uploading all the files:

In [24]:
# File paths
stage_path = "@tpcdi/Batch1/FINWIRE"

# glob the files
pathlist = (
    Path(source_path)
    .glob("FINWIRE??????")
)

for file in pathlist:
    # put the file(s) in the stage
    put_result = (
        session 
        .file
        .put(
            str(file), 
            stage_path, 
            parallel=4, 
            auto_compress=True
        )
    )
    for result in put_result:
        print(f"File {result.source}: {result.status}")

File FINWIRE2001Q3: SKIPPED
File FINWIRE2001Q4: SKIPPED
File FINWIRE1997Q4: SKIPPED
File FINWIRE1970Q3: SKIPPED
File FINWIRE1997Q3: SKIPPED
File FINWIRE1970Q4: SKIPPED
File FINWIRE1999Q2: SKIPPED
File FINWIRE1991Q1: SKIPPED
File FINWIRE2005Q2: SKIPPED
File FINWIRE1976Q1: SKIPPED
File FINWIRE1993Q2: SKIPPED
File FINWIRE1974Q2: SKIPPED
File FINWIRE2007Q1: SKIPPED
File FINWIRE1997Q2: SKIPPED
File FINWIRE1978Q1: SKIPPED
File FINWIRE1970Q2: SKIPPED
File FINWIRE2003Q1: SKIPPED
File FINWIRE1995Q1: SKIPPED
File FINWIRE2009Q1: SKIPPED
File FINWIRE2001Q2: SKIPPED
File FINWIRE1972Q1: SKIPPED
File FINWIRE1974Q3: SKIPPED
File FINWIRE1993Q4: SKIPPED
File FINWIRE1974Q4: SKIPPED
File FINWIRE1993Q3: SKIPPED
File FINWIRE1999Q4: SKIPPED
File FINWIRE2005Q3: SKIPPED
File FINWIRE2005Q4: SKIPPED
File FINWIRE1999Q3: SKIPPED
File FINWIRE1987Q2: SKIPPED
File FINWIRE1968Q1: SKIPPED
File FINWIRE2013Q1: SKIPPED
File FINWIRE1985Q1: SKIPPED
File FINWIRE2011Q2: SKIPPED
File FINWIRE1983Q4: SKIPPED
File FINWIRE1983Q3: 

KeyboardInterrupt: 

The CMP, SEC, and FIN records all have two fields in common, so we want to create a generic DataFrame that contains the shared logic and we’ll save that DataFrame as a Snowflake temporary table called FINWIRE:

In [20]:
# These are fixed-width fields, so read the entire line in as "line"
schema = StructType([
        StructField("line", StringType(), False),
])

# generic dataframe for all record types
# create a temporary table
# The delimiter '|' seems safer
df = (
    session
    .read
    .schema(schema)
    .option('field_delimiter', '|')
    .csv(stage_path)
    .with_column(
        'pts', 
        to_timestamp(
            substring(col("line"), lit(0), lit(15)), 
            lit("yyyymmdd-hhmiss")
        )
    )
    .with_column(
        'rec_type', 
        substring(col("line"), lit(16), lit(3))
    )
    .write
    .mode("overwrite")
    .save_as_table("finwire", table_type="temporary")
)

# let's see the table
df = (
    session 
    .table('finwire') 
    .show()
)

-----------------------------------------------------------------------------------------
|"LINE"                                              |"PTS"                |"REC_TYPE"  |
-----------------------------------------------------------------------------------------
|19670401-065923FIN196721967040119670401    9288...  |1967-04-01 06:59:23  |FIN         |
|19670401-161220FIN196721967040119670401    6180...  |1967-04-01 16:12:20  |FIN         |
|19670402-012108FIN196721967040119670402     818...  |1967-04-02 01:21:08  |FIN         |
|19670402-140519FIN196721967040119670402    3590...  |1967-04-02 14:05:19  |FIN         |
|19670403-051650FIN196721967040119670403    6457...  |1967-04-03 05:16:50  |FIN         |
|19670403-194201FIN196721967040119670403    6692...  |1967-04-03 19:42:01  |FIN         |
|19670404-011711FIN196721967040119670404    5352...  |1967-04-04 01:17:11  |FIN         |
|19670404-023010FIN196721967040119670404    7901...  |1967-04-04 02:30:10  |FIN         |
|19670404-

Now I can create the three separate tables from this temporary table using `WITH_COLUMN` and `SUBSTRING`.

I'll only show the Security table as an example, but the other two are done the same way:

In [22]:
# SEC record types
table_name = 'sec'
df = (
    session
    .table('finwire')
    .where(col('rec_type') == 'SEC')
    .withColumn(
        'symbol', 
        substring(col("line"), lit(19), lit(15))
    )
    .withColumn(
        'issue_type', 
        substring(col("line"), lit(34), lit(6))
    )
    .withColumn(
        'status', 
        substring(col("line"), lit(40), lit(4))
    )
    .withColumn(
        'name', 
        substring(col("line"), lit(44), lit(70))
    )
    .withColumn(
        'ex_id', 
        substring(col("line"), lit(114), lit(6))
    )
    .withColumn(
        'sh_out', 
        substring(col("line"), lit(120), lit(13))
    )
    .withColumn(
        'first_trade_date', 
        substring(col("line"), lit(133), lit(8))
    )
    .withColumn(
        'first_exchange_date', 
        substring(col("line"), lit(141), lit(8))
    )
    .withColumn(
        'dividend', 
        substring(col("line"), lit(149), lit(12))
    )
    .withColumn(
        'co_name_or_cik', 
        substring(col("line"), lit(161), lit(60))
    )
    .drop(col("line"), col("rec_type"))
    .write
    .mode("overwrite")
    .save_as_table(table_name)
)

print(f"{table_name.upper()} table created.")

# let's see the table
df = (
    session 
    .table('sec') 
    .show()
)

SEC table created.
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PTS"                |"SYMBOL"         |"ISSUE_TYPE"  |"STATUS"  |"NAME"                                              |"EX_ID"  |"SH_OUT"       |"FIRST_TRADE_DATE"  |"FIRST_EXCHANGE_DATE"  |"DIVIDEND"    |"CO_NAME_OR_CIK"                                    |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1968-06-12 02:43:03  |AAAAAAAAAAAAAJG  |COMMON        |ACTV      |rFALDSWBSGSnzzMwTwjF                           ...  |PCX      |792341095      |19230923            |19301212               |        0.

The `DIGen.jar` utility creates a single XML called `CustomerMgmt.xml`, with a sample below:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<TPCDI:Actions xmlns:TPCDI="http://www.tpc.org/tpc-di">
 <TPCDI:Action ActionType="NEW" ActionTS="2007-07-07T02:56:25">
  <Customer C_ID="0" C_TAX_ID="923-54-6498" C_GNDR="F" C_TIER="3" C_DOB="1940-12-02">
   <Name>
    <C_L_NAME>Joannis</C_L_NAME>
    <C_F_NAME>Adara</C_F_NAME>
    <C_M_NAME/>
   </Name>
   <Address>
    <C_ADLINE1>4779 Weller Way</C_ADLINE1>
    <C_ADLINE2/>
    <C_ZIPCODE>92624</C_ZIPCODE>
    <C_CITY>Columbus</C_CITY>
    <C_STATE_PROV>Ontario</C_STATE_PROV>
    <C_CTRY>Canada</C_CTRY>
   </Address>
   <ContactInfo>
    <C_PRIM_EMAIL>Adara.Joannis@moose-mail.com</C_PRIM_EMAIL>
    <C_ALT_EMAIL>Adara.Joannis@gmx.com</C_ALT_EMAIL>
    <C_PHONE_1>
     <C_CTRY_CODE>1</C_CTRY_CODE>
     <C_AREA_CODE>872</C_AREA_CODE>
     <C_LOCAL>523-8928</C_LOCAL>
     <C_EXT/>
    </C_PHONE_1>
    <C_PHONE_2>
     <C_CTRY_CODE/>
     <C_AREA_CODE/>
     <C_LOCAL>492-3961</C_LOCAL>
     <C_EXT/>
    </C_PHONE_2>
    <C_PHONE_3>
     <C_CTRY_CODE/>
     <C_AREA_CODE/>
     <C_LOCAL/>
     <C_EXT/>
    </C_PHONE_3>
   </ContactInfo>
   <TaxInfo>
    <C_LCL_TX_ID>CA3</C_LCL_TX_ID>
    <C_NAT_TX_ID>YT3</C_NAT_TX_ID>
   </TaxInfo>
   <Account CA_ID="0" CA_TAX_ST="1">
    <CA_B_ID>17713</CA_B_ID>
    <CA_NAME>CJlmMuFyibKOmKLHIaTeWugvCgZdmcfpDsYb</CA_NAME>
   </Account>
  </Customer>
 </TPCDI:Action>
</TPCDI:Actions>
```

The hierarchical representation of a TPCDI:Action record, with @ signifying a node attribute as opposed to an element, is shown below:

```
|-- TPCDI:Action
    |-- @ActionType: string
    |-- @ActionTS: timestamp
    |-- Customer
        |-- @C_ID: number
        |-- @C_TAX_ID: string
        |-- @C_GNDR: string
        |-- @C_TIER: number
        |-- @C_DOB: date
        |-- Name
            |-- C_F_NAME: string
            |-- C_L_NAME: string
            |-- C_M_NAME: string
        |-- Address
            |-- C_ADLINE1: string
            |-- C_ADLINE2: string
            |-- C_CITY: string
            |-- C_CTRY: string
            |-- C_STATE_PROV: string
            |-- C_ZIPCODE: string
        |-- ContactInfo
            |-- C_ALT_EMAIL: string
            |-- C_PHONE_1
                |-- C_AREA_CODE: number
                |-- C_CTRY_CODE: number
                |-- C_EXT: long
                |-- C_LOCAL: string
            |-- C_PHONE_2
                |-- C_AREA_CODE: number
                |-- C_CTRY_CODE: number
                |-- C_EXT: number
                |-- C_LOCAL: string
            |-- C_PHONE_3
                |-- C_AREA_CODE: number
                |-- C_CTRY_CODE: number
                |-- C_EXT: number
                |-- C_LOCAL: string
        |-- TaxInfo
            |-- C_LCL_TX_ID: string
            |-- C_NAT_TX_ID: string
        |-- Account
            |-- CA_B_ID: number
            |-- CA_NAME: string
            |-- @CA_ID: number
            |-- @CA_TAX_ST: number
```


Let's create a DataFrame from the XML file and see what we get:

In [26]:
# File paths
stage_path = "@tpcdi/Batch1"

# Put the file
put_result = (
    session
    .file
    .put(
        f"{source_path}/CustomerMgmt.xml",
        f"{stage_path}/CustomerMgmt.xml",
        parallel=4,
        auto_compress=True,
    )
)
for result in put_result:
    print(f"File {result.source}: {result.status}")

# Read the XML file into a DataFrame and show it
df = (
    session
    .read
    .option('STRIP_OUTER_ELEMENT', True) # Strips TPCDI:Actions
    .xml(f"{stage_path}/CustomerMgmt.xml")
    .show(1, 100)
)

File CustomerMgmt.xml: SKIPPED
-----------------------------------------------------------------------------------------
|"$1"                                                                                   |
-----------------------------------------------------------------------------------------
|<TPCDI:Action ActionTS="2007-07-07T04:28:56" ActionType="NEW">                         |
|  <Customer C_DOB="1940-12-02" C_GNDR="F" C_ID="0" C_TAX_ID="923-54-6498" C_TIER="3">  |
|    <Name>                                                                             |
|      <C_L_NAME>Joannis</C_L_NAME>                                                     |
|      <C_F_NAME>Adara</C_F_NAME>                                                       |
|      <C_M_NAME></C_M_NAME>                                                            |
|    </Name>                                                                            |
|    <Address>                                                       

Snowflake does not support simple dot notation for XML the way it does for JSON.

Instead we have to pair the `GET()` function with an `XMLGET()`, which can be quite tedious.

So I wrote a few helper functions to encapsulate that:

In [27]:
# Simplifies retrieving XML elements
def get_xml_element(
        column:str,
        element:str,
        datatype:str,
        with_alias:bool = True
):
    new_element = (
        get(
            xmlget(
                col(column),
                lit(element),
            ),
            lit('$')
        )
        .cast(datatype)
    )

    # alias needs to be optional
    return (
        new_element.alias(element) if with_alias else new_element
    )

# Simplifies retrieving XML attributes
def get_xml_attribute(
        column:str,
        attribute:str,
        datatype:str,
        with_alias:bool = True
):
    new_attribute = (
        get(
            col(column),
            lit(f"@{attribute}")
        )
        .cast(datatype)
    )

    # alias needs to be optional
    return (
        new_attribute.alias(attribute) if with_alias else new_attribute
    )

# Constructs a phone number from multiple nested fields
def get_phone_number(
        phone_id:str,
        separator:str = '-'
):
    return (
        concat (
            get_xml_element(f"phone{phone_id}", 'C_CTRY_CODE', 'STRING', False),
            when(get_xml_element(f"phone{phone_id}", 'C_CTRY_CODE', 'STRING', False) == '', '')
            .otherwise(separator),
            get_xml_element(f"phone{phone_id}", 'C_AREA_CODE', 'STRING', False),
            when(get_xml_element(f"phone{phone_id}", 'C_AREA_CODE', 'STRING', False) == '', '')
            .otherwise(separator),
            get_xml_element(f"phone{phone_id}", 'C_LOCAL', 'STRING', False),
            when(get_xml_element(f"phone{phone_id}", 'C_EXT', 'STRING', False) == '', '')
            .otherwise(" ext: "),
            get_xml_element(f"phone{phone_id}", 'C_EXT', 'STRING', False)
        )
        .alias(f"c_phone_{phone_id}")
    )

Now let's put it all together and create our `customer_mgmt` table:

In [30]:
table_name = 'customer_mgmt'
df = (
    session
    .read
    .option('STRIP_OUTER_ELEMENT', True) # Strips the TPCDI:Actions node
    .xml(f"{stage_path}/CustomerMgmt.xml")
    .select(
        # flatten out all of the nested elements
        xmlget(col('$1'), lit('Customer'), 0).alias('customer'),
        xmlget(col('customer'), lit('Name'), 0).alias('name'),
        xmlget(col('customer'), lit('Address'), 0).alias('address'),
        xmlget(col('customer'), lit('ContactInfo'), 0).alias('contact_info'),
        xmlget(col('contact_info'), lit('C_PHONE_1')).alias('phone1'),
        xmlget(col('contact_info'), lit('C_PHONE_2')).alias('phone2'),
        xmlget(col('contact_info'), lit('C_PHONE_3')).alias('phone3'),
        xmlget(col('customer'), lit('TaxInfo'), 0).alias('tax_info'),
        xmlget(col('customer'), lit('Account'), 0).alias('account'),
        # get the Action attributes
        get_xml_attribute('$1','ActionType','STRING'),
        get_xml_attribute('$1','ActionTS','STRING'),
    )
    .select(
        # Handling Action attributes
        to_timestamp(
            col('ActionTs'),
            lit('yyyy-mm-ddThh:mi:ss')
        ).alias('action_ts'),
        col('ActionType').alias('ACTION_TYPE'),
        # Get Customer Attributes
        get_xml_attribute('customer','C_ID','NUMBER'),
        get_xml_attribute('customer','C_TAX_ID','STRING'),
        get_xml_attribute('customer','C_GNDR','STRING'),
        # Had to disable auto-aliasing
        try_cast(
            get_xml_attribute('customer','C_TIER','STRING', False),
            'NUMBER'
        ).alias('c_tier'),
        get_xml_attribute('customer','C_DOB','DATE'),
        # Get Name elements
        get_xml_element('name','C_L_NAME','STRING'),
        get_xml_element('name','C_F_NAME','STRING'),
        get_xml_element('name','C_M_NAME','STRING'),
        # Get Address elements
        get_xml_element('address','C_ADLINE1','STRING'),
        get_xml_element('address', 'C_ADLINE2', 'STRING'),
        get_xml_element('address','C_ZIPCODE','STRING'),
        get_xml_element('address','C_CITY','STRING'),
        get_xml_element('address','C_STATE_PROV','STRING'),
        get_xml_element('address','C_CTRY','STRING'),
        # Get Contact Info elements
        get_xml_element('contact_info','C_PRIM_EMAIL','STRING'),
        get_xml_element('contact_info','C_ALT_EMAIL','STRING'),
        # Contruct phone numbers from multi-nested elements
        get_phone_number('1'),
        get_phone_number('2'),
        get_phone_number('3'),
        # Get TaxInfo elements
        get_xml_element('tax_info','C_LCL_TX_ID','STRING'),
        get_xml_element('tax_info','C_NAT_TX_ID','STRING'),
        # Get Account Attributes
        get_xml_attribute('account','CA_ID','STRING'),
        get_xml_attribute('account','CA_TAX_ST','NUMBER'),
        # Get Account elements
        get_xml_element('account','CA_B_ID','NUMBER'),
        get_xml_element('account','CA_NAME','STRING'),
    )
    .write
    .mode("overwrite")
    .save_as_table(table_name)
)

print(f"{table_name.upper()} table created.")

df = (
    session
    .table('customer_mgmt')
    .select(
        col('action_ts'),
        col('c_id'),
        col('c_tier'),
        col('c_phone_1')
    )
    .show()
)

CUSTOMER_MGMT table created.
---------------------------------------------------------------------
|"ACTION_TS"          |"C_ID"  |"C_TIER"  |"C_PHONE_1"              |
---------------------------------------------------------------------
|2007-07-07 04:28:56  |0       |3         |1-872-523-8928           |
|2007-07-07 04:47:03  |1       |3         |767-4707                 |
|2007-07-07 06:17:28  |2       |3         |420-757-3642 ext: 61998  |
|2007-07-07 07:57:28  |3       |3         |1-819-163-0774           |
|2007-07-07 09:38:29  |4       |NULL      |734-4072                 |
|2007-07-07 11:35:54  |5       |3         |667-588-0328             |
|2007-07-07 15:00:43  |6       |3         |1-475-246-3524           |
|2007-07-07 21:05:32  |7       |3         |466-5901                 |
|2007-07-08 00:32:37  |8       |NULL      |540-4805                 |
|2007-07-08 04:14:53  |9       |NULL      |230-135-8787             |
-------------------------------------------------------------

![ETL Diagram](images/tpc-di-logical-model.png)

# When we Google "dbt dynamic tables":

![Google Search](images/dbt-dynamic-tables.png)

# It's not as simple as this.

![Conflict](images/refresh-conflict.png)

# dbt is more than just a job scheduler.
Dynamic Tables need to be (re)created in the correct order. This can become very complex as the number of tables and dependencies increases.

dbt understands your DAG.

In [31]:
!dbt docs generate
!dbt docs serve

21:23:19  Running with dbt=1.7.2
21:23:19  Registered adapter: snowflake=1.7.0
21:23:19  Found 45 models, 17 sources, 0 exposures, 0 metrics, 544 macros, 0 groups, 0 semantic models
21:23:19  
21:23:22  Concurrency: 20 threads (target='dev')
21:23:22  
21:23:22  Building catalog
21:23:29  Catalog written to /Users/stewartbryson/Source/dbt-tpcdi/target/catalog.json
21:23:30  Running with dbt=1.7.2
Serving docs at 8080
To access from your browser, navigate to: http://localhost:8080



Press Ctrl+C to exit.
127.0.0.1 - - [27/Nov/2023 16:23:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2023 16:23:31] "GET /manifest.json?cb=1701120211638 HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2023 16:23:31] "GET /catalog.json?cb=1701120211638 HTTP/1.1" 200 -
^C
21:24:08  Encountered an error:

21:24:08  Traceback (most recent call last):
  File "/Users/stewartbryson/anaconda3/envs/tpcdi/lib/python3.8/site-packages/dbt/cli/requires.py", line 90, in wrapper
    result, success = func(*args, **kwargs)
  File

### We can see all that's required to enable dynamic tables in our `dbt_project.yml` file:

```yaml
models:
  dbt_tpcdi:
    example:
      +materialized: view
    bronze:
      +schema: bronze
      +materialized: dynamic_table
      +snowflake_warehouse: tpcdi_large
      +target_lag: downstream
    silver:
      +schema: silver
      +materialized: dynamic_table
      +snowflake_warehouse: tpcdi_large
      +target_lag: '10 minutes'
    gold:
      +schema: gold
      +materialized: dynamic_table
      +snowflake_warehouse: tpcdi_large
      +target_lag: '20 minutes'
    work:
      +schema: work
      +materialized: ephemeral
```

In [32]:
!dbt build

21:26:43  Running with dbt=1.7.2
21:26:43  Registered adapter: snowflake=1.7.0
21:26:43  Found 45 models, 17 sources, 0 exposures, 0 metrics, 544 macros, 0 groups, 0 semantic models
21:26:43  
21:26:45  Concurrency: 20 threads (target='dev')
21:26:45  
21:26:45  1 of 44 START sql dynamic_table model dl_bronze.brokerage_cash_transaction ..... [RUN]
21:26:45  2 of 44 START sql dynamic_table model dl_bronze.brokerage_daily_market ......... [RUN]
21:26:45  3 of 44 START sql dynamic_table model dl_bronze.brokerage_holding_history ...... [RUN]
21:26:45  4 of 44 START sql dynamic_table model dl_bronze.brokerage_trade ................ [RUN]
21:26:45  5 of 44 START sql dynamic_table model dl_bronze.brokerage_trade_history ........ [RUN]
21:26:45  6 of 44 START sql dynamic_table model dl_bronze.brokerage_watch_history ........ [RUN]
21:26:45  7 of 44 START sql dynamic_table model dl_bronze.crm_customer_mgmt .............. [RUN]
21:26:45  8 of 44 START sql dynamic_table model dl_bronze.finwire_co

Click this link to open results:

[Snowflake UI](https://app.snowflake.com/cxmdykz/hib36835/#/data/databases/TPCDI)

### dbt also has Tests.

We can run them when we create the Dynamic Table:

In [33]:
!dbt build --select fact_trade

21:28:43  Running with dbt=1.7.2
21:28:44  Registered adapter: snowflake=1.7.0
21:28:44  Found 45 models, 17 sources, 0 exposures, 0 metrics, 544 macros, 0 groups, 0 semantic models
21:28:44  
21:28:45  Concurrency: 20 threads (target='dev')
21:28:45  
21:28:45  1 of 1 START sql dynamic_table model dl_gold.fact_trade ........................ [RUN]
21:28:48  1 of 1 OK created sql dynamic_table model dl_gold.fact_trade ................... [SUCCESS 1 in 2.30s]
21:28:48  
21:28:48  Finished running 1 dynamic_table model in 0 hours 0 minutes and 3.74 seconds (3.74s).
21:28:48  
21:28:48  Completed successfully
21:28:48  
21:28:48  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1


# Or we can schedule them to run periodically:

In [35]:
!dbt test

21:29:25  Running with dbt=1.7.2
21:29:25  Registered adapter: snowflake=1.7.0
21:29:25  Found 45 models, 1 test, 17 sources, 0 exposures, 0 metrics, 544 macros, 0 groups, 0 semantic models
21:29:25  
21:29:26  Concurrency: 20 threads (target='dev')
21:29:26  
21:29:26  1 of 1 START test fact_trade__unique_trade ..................................... [RUN]
21:29:27  1 of 1 PASS fact_trade__unique_trade ........................................... [PASS in 1.39s]
21:29:27  
21:29:27  Finished running 1 test in 0 hours 0 minutes and 2.03 seconds (2.03s).
21:29:27  
21:29:27  Completed successfully
21:29:27  
21:29:27  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1


# More than just a scheduler.

1. Cloud development environment for those that prefer it.
1. CI/CD workflows for promoting Dynamic Table changes into Production.
1. Perhaps there's promise in the Semantic Layer.

# Clean-up

In [36]:
!python tpcdi.py drop-schema --schema dl_gold
!python tpcdi.py drop-schema --schema dl_silver
!python tpcdi.py drop-schema --schema dl_bronze
!python tpcdi.py drop-schema --schema dl_work

Schema dl_gold dropped.
Schema dl_silver dropped.
Schema dl_bronze dropped.
Schema dl_work dropped.
